In [0]:
!git clone https://github.com/MaurizioFD/RecSys_Course_2018.git
!mv -v ./RecSys_Course_2018/* ./
!rm -r ./RecSys_Course_2018/
!pip install -r requirements.txt
!python run_compile_all_cython.py

Cloning into 'RecSys_Course_2018'...
remote: Enumerating objects: 252, done.
remote: Counting objects: 100% (252/252), done.
remote: Compressing objects: 100% (178/178), done.
remote: Total 676 (delta 118), reused 171 (delta 71), pack-reused 424
Receiving objects: 100% (676/676), 34.44 MiB | 21.27 MiB/s, done.
Resolving deltas: 100% (312/312), done.
renamed './RecSys_Course_2018/Base' -> './Base'
renamed './RecSys_Course_2018/Basics-1-Introduction-to-Python-Programming.ipynb' -> './Basics-1-Introduction-to-Python-Programming.ipynb'
renamed './RecSys_Course_2018/Basics-2-Numpy.ipynb' -> './Basics-2-Numpy.ipynb'
renamed './RecSys_Course_2018/Basics-3-Scipy.ipynb' -> './Basics-3-Scipy.ipynb'
renamed './RecSys_Course_2018/CythonCompiler' -> './CythonCompiler'
renamed './RecSys_Course_2018/Data_manager' -> './Data_manager'
renamed './RecSys_Course_2018/FeatureWeighting' -> './FeatureWeighting'
renamed './RecSys_Course_2018/GraphBased' -> './GraphBased'
renamed './RecSys_Course_2018/KNN' -> 

run_compile_all_cython: Found 3 Cython files in 3 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/usr/bin/python3'
Compiling [1/3]: MatrixFactorization_Cython_Epoch.pyx... 
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/MatrixFactorization/Cython/MatrixFactorization_Cython_Epoch.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
In file included from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1822:0,
                 from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from MatrixFactorization_Cython_Epoch.c:593:
/usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/npy_1_7_dep

In [0]:
URM_file = open("data/data_train.csv", 'r')

lines = URM_file.readline()

def rowSplit (rowString):
    split = rowString.split(",")
    split[2] = split[2].replace("\n", " ")

    split[0] = int(split[0])
    split[1] = int(split[1])
    split[2] = float(split[2])

    result = tuple(split)

    return result


URM_file.seek(0)
URM_tuples = []

for line in URM_file:
   URM_tuples.append(rowSplit(line))

userList, itemList, ratingList = zip(*URM_tuples)

userList = list(userList)
itemList = list(itemList)
ratingList = list(ratingList)

userList_unique = list(set(userList))
itemList_unique = list(set(itemList))
numUsers = len(userList_unique)
numItems = len(itemList_unique)

import scipy.sparse as sps
URM_all = sps.coo_matrix((ratingList, (userList, itemList)))
URM_all = URM_all.tocsr()

import numpy as np
import scipy.sparse as sps

def train_test_holdout(URM_all, train_perc = 0.8):


    numInteractions = URM_all.nnz
    URM_all = URM_all.tocoo()
    shape = URM_all.shape


    train_mask = np.random.choice([True,False], numInteractions, p=[train_perc, 1-train_perc])


    URM_train = sps.coo_matrix((URM_all.data[train_mask], (URM_all.row[train_mask], URM_all.col[train_mask])), shape=shape)
    URM_train = URM_train.tocsr()

    test_mask = np.logical_not(train_mask)

    URM_test = sps.coo_matrix((URM_all.data[test_mask], (URM_all.row[test_mask], URM_all.col[test_mask])), shape=shape)
    URM_test = URM_test.tocsr()

    return URM_train, URM_test

URM_train, URM_test = train_test_holdout(URM_all, train_perc=0.8)

In [0]:
from ParameterTuning.run_parameter_search import *

In [0]:
from Base.Evaluation import *
from Base import *


In [0]:
collaborative_algorithm_list = [
        Random,
        TopPop,
        P3alphaRecommender,
        RP3betaRecommender,
        ItemKNNCFRecommender,
        UserKNNCFRecommender,
        MatrixFactorization_BPR_Cython,
        MatrixFactorization_FunkSVD_Cython,
        PureSVDRecommender,
        SLIM_BPR_Cython,
        SLIMElasticNetRecommender
    ]

In [0]:
from Base.Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])
    

In [0]:
runParameterSearch_Collaborative(SLIM_BPR_Cython,
                                 URM_train=URM_train,
                                 n_cases = 10,
                                 metric_to_optimize = "MAP",
                                 evaluator_validation_earlystopping = evaluator_validation,
                                 evaluator_validation = evaluator_validation
                                 )

Iteration No: 1 started. Evaluating function at random point.
SLIM_BPR_Recommender: URM Detected 4284 (13.86 %) cold users.
SLIM_BPR_Recommender: URM Detected 3714 (20.08 %) cold items.
SearchBayesianSkopt: Testing config: {'topK': 167, 'epochs': 1500, 'symmetric': False, 'sgd_mode': 'adam', 'lambda_i': 3.876366200055673e-05, 'lambda_j': 0.008745889319969152, 'learning_rate': 0.09133243317194147}
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Available RAM is 12089.00 MB (92.84%) of 13022.00 MB, required is 2736.52 MB. Using dense matrix.
Processed 30911 ( 100.00% ) in 0.77 seconds. BPR loss is 2.02E+00. Sample per second: 40242
SLIM_BPR_Recommender: Epoch 1 of 1500. Elapsed time 0.03 sec
Processed 30911 ( 100.00% ) in 0.81 seconds. BPR loss is 9.21E+00. Sample per second: 38073
SLIM_BPR_Recommender: Epoch 2 of 1500. Elapsed time 0.08 sec
Processed 30911 ( 100.00% ) in 0.85 seconds. BPR loss is 1.37E+01. Sample per second: 36370
SLIM_BPR_Recommender: Epoch 3 of 1500. 

In [0]:
runParameterSearch_Collaborative(MatrixFactorization_BPR_Cython,
                                 URM_train=URM_train,
                                 n_cases = 8,
                                 metric_to_optimize = "MAP",
                                 evaluator_validation_earlystopping = evaluator_validation,
                                 evaluator_validation = evaluator_validation
                                 )

In [0]:
#Testing config: {'topK': 5, 'epochs': 1500, 'symmetric': True, 'sgd_mode': 'adagrad', 'lambda_i': 1e-05, 'lambda_j': 1e-05, 'learning_rate': 0.0001} =0.0406

#============================

In [40]:

from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython

recommender = SLIM_BPR_Cython(URM_train)
recommender.fit(epochs=4000, topK=10, symmetric=True, sgd_mode='adagrad', lambda_i=1e-05, lambda_j= 1e-05, learning_rate=0.000001)

SLIM_BPR_Recommender: URM Detected 4320 (13.98 %) cold users.
SLIM_BPR_Recommender: URM Detected 3722 (20.12 %) cold items.
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Available RAM is 12131.00 MB (93.16%) of 13022.00 MB, required is 1368.26 MB. Using dense matrix.
Processed 30911 ( 100.00% ) in 0.25 seconds. BPR loss is 5.01E-12. Sample per second: 122526
SLIM_BPR_Recommender: Epoch 1 of 4000. Elapsed time 0.07 sec
Processed 30911 ( 100.00% ) in 0.32 seconds. BPR loss is 1.35E-11. Sample per second: 97216
SLIM_BPR_Recommender: Epoch 2 of 4000. Elapsed time 0.13 sec
Processed 30911 ( 100.00% ) in 0.38 seconds. BPR loss is 2.20E-11. Sample per second: 81017
SLIM_BPR_Recommender: Epoch 3 of 4000. Elapsed time 0.20 sec
Processed 30911 ( 100.00% ) in 0.44 seconds. BPR loss is 2.73E-11. Sample per second: 69468
SLIM_BPR_Recommender: Epoch 4 of 4000. Elapsed time 0.26 sec
Processed 30911 ( 100.00% ) in 0.51 seconds. BPR loss is 3.52E-11. Sample per second: 61017
SLIM_BPR

In [0]:
import numpy as np
import scipy.sparse as sps



def precision(is_relevant, relevant_items):

    #is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)

    return precision_score



def recall(is_relevant, relevant_items):

    #is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]

    return recall_score



def MAP(is_relevant, relevant_items):

    #is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    map_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score



def evaluate_algorithm_for_elastic(URM_test, recommender_object, at=10):

    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_MAP = 0.0

    num_eval = 0

    URM_test = sps.csr_matrix(URM_test)

    n_users = URM_test.shape[0]


    for user_id in range(n_users):

        if user_id % 10000 == 0:
            print("Evaluated user {} of {}".format(user_id, n_users))

        start_pos = URM_test.indptr[user_id]
        end_pos = URM_test.indptr[user_id+1]

        if end_pos-start_pos>0:

            relevant_items = URM_test.indices[start_pos:end_pos]

            recommended_items = recommender_object.recommend(user_id)[:10]
            num_eval+=1

            is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

            cumulative_precision += precision(is_relevant, relevant_items)
            cumulative_recall += recall(is_relevant, relevant_items)
            cumulative_MAP += MAP(is_relevant, relevant_items)


    cumulative_precision /= num_eval
    cumulative_recall /= num_eval
    cumulative_MAP /= num_eval

    print("Recommender performance is: Precision = {:.4f}, Recall = {:.4f}, MAP = {:.4f}".format(
        cumulative_precision, cumulative_recall, cumulative_MAP))

    result_dict = {
        "precision": cumulative_precision,
        "recall": cumulative_recall,
        "MAP": cumulative_MAP,
    }

    return result_dict

In [41]:
from Notebooks_utils.evaluation_function import evaluate_algorithm 
result_dict = evaluate_algorithm_for_elastic( URM_test, recommender )
print( result_dict["MAP"] )

Evaluated user 0 of 30911
Evaluated user 10000 of 30911
Evaluated user 20000 of 30911
Evaluated user 30000 of 30911
Recommender performance is: Precision = 0.0265, Recall = 0.0775, MAP = 0.0423
0.04225109419759558


In [0]:
#recommender.fit(epochs=1500, topK=10, symmetric=True, sgd_mode='adagrad', lambda_i=1e-05, lambda_j= 1e-05, learning_rate=0.000001) == 0.0415
3000==0.0423
4500==0.0423

recommender.fit(epochs=1500, topK=5, symmetric=True, sgd_mode='adagrad', lambda_i=1e-05, lambda_j= 1e-05, learning_rate=0.0001) ==0.0410


In [0]:
import numpy as np

class TopPopRecommender( object ):

    def fit(self, URM_train):
        itemPopularity = (URM_train > 0).sum( axis=0 )
        itemPopularity = np.array( itemPopularity ).squeeze()

        # We are not interested in sorting the popularity value,
        # but to order the items according to it
        self.popularItems = np.argsort( itemPopularity )
        self.popularItems = np.flip( self.popularItems, axis=0 )

    def recommend(self, user_id, at=5):
        recommended_items = self.popularItems[0:at]

        return recommended_items

topPopRecommender = TopPopRecommender()
topPopRecommender.fit(URM_all)
TopTen = []

for user_id in userList_unique:
    TopTen = topPopRecommender.recommend(user_id, at=10)

In [32]:
from KNN.ItemKNNCFRecommender import *

itemKNNRecommender = ItemKNNCFRecommender(URM_train)
itemKNNRecommender.fit()

ItemKNNCFRecommender: URM Detected 4320 (13.98 %) cold users.
ItemKNNCFRecommender: URM Detected 3722 (20.12 %) cold items.
Similarity column 18495 ( 100 % ), 9179.51 column/sec, elapsed time 0.03 min


In [0]:
class Mix_recommender(object):
    def __init__(self,r1,r2,bpr,weight):
        self.r1 = r1
        self.r2 = r2
        self.bpr = bpr
        self.weight = weight
    def recommend(self,uid,at):
        items = []
        items += list(self.r1.recommend(uid,at=self.weight[0]))
        items += list(self.r2.recommend(uid)[:self.weight[1]])
        items += list(self.bpr.recommend(uid)[:self.weight[2]])
        return items


In [48]:
mix_recommender = Mix_recommender(topPopRecommender,itemKNNRecommender,recommender,[0,3,7])
from Notebooks_utils.evaluation_function import evaluate_algorithm
result_dict = evaluate_algorithm(URM_test, mix_recommender)

Evaluated user 0 of 30911
Evaluated user 200 of 30911
Evaluated user 400 of 30911
Evaluated user 600 of 30911
Evaluated user 800 of 30911
Evaluated user 1000 of 30911
Evaluated user 1200 of 30911
Evaluated user 1400 of 30911
Evaluated user 1600 of 30911
Evaluated user 1800 of 30911
Evaluated user 2000 of 30911
Evaluated user 2200 of 30911
Evaluated user 2400 of 30911
Evaluated user 2600 of 30911
Evaluated user 2800 of 30911
Evaluated user 3000 of 30911
Evaluated user 3200 of 30911
Evaluated user 3400 of 30911
Evaluated user 3600 of 30911
Evaluated user 3800 of 30911
Evaluated user 4000 of 30911
Evaluated user 4200 of 30911
Evaluated user 4400 of 30911
Evaluated user 4600 of 30911
Evaluated user 4800 of 30911
Evaluated user 5000 of 30911
Evaluated user 5200 of 30911
Evaluated user 5400 of 30911
Evaluated user 5600 of 30911
Evaluated user 5800 of 30911
Evaluated user 6000 of 30911
Evaluated user 6200 of 30911
Evaluated user 6400 of 30911
Evaluated user 6600 of 30911
Evaluated user 6800 o

[top,itemKNN,Bpr]=[1:3:6]==0.0461

[0,4,6]==0.0662

0.0673

0.0674
[0,3,7]=0.0683

In [46]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from KNN.ItemKNNCFRecommender import *

r2 = ItemKNNCFRecommender(URM_all)
r2.fit()

r3 = SLIM_BPR_Cython(URM_all)
r3.fit(epochs=4500, topK=10, symmetric=True, sgd_mode='adagrad', lambda_i=1e-05, lambda_j= 1e-05, learning_rate=0.000001)

mix = Mix_recommender(topPopRecommender,r2,r3,[0,3,7])

ItemKNNCFRecommender: URM Detected 3656 (11.83 %) cold users.
ItemKNNCFRecommender: URM Detected 3218 (17.40 %) cold items.
Similarity column 18495 ( 100 % ), 8678.71 column/sec, elapsed time 0.04 min
SLIM_BPR_Recommender: URM Detected 3656 (11.83 %) cold users.
SLIM_BPR_Recommender: URM Detected 3218 (17.40 %) cold items.
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Available RAM is 12140.00 MB (93.23%) of 13022.00 MB, required is 1368.26 MB. Using dense matrix.
Processed 30911 ( 100.00% ) in 0.18 seconds. BPR loss is 8.49E-12. Sample per second: 169162
SLIM_BPR_Recommender: Epoch 1 of 4500. Elapsed time 0.07 sec
Processed 30911 ( 100.00% ) in 0.25 seconds. BPR loss is 2.18E-11. Sample per second: 122502
SLIM_BPR_Recommender: Epoch 2 of 4500. Elapsed time 0.14 sec
Processed 30911 ( 100.00% ) in 0.32 seconds. BPR loss is 3.51E-11. Sample per second: 96677
SLIM_BPR_Recommender: Epoch 3 of 4500. Elapsed time 0.21 sec


KeyboardInterrupt: ignored

In [0]:
cold_To_warm_users ={}
with open('/content/data/cold_user_similarity.csv') as f:
  line = f.readline()
  while line:
    line.replace('\n','')
    split = line.split(',')
    cold_To_warm_users.update({int(split[0]):int(split[1])})
    line = f.readline()

cold_users = []
with open('/content/data/cold_users') as f:
  line = f.readline()
  while line:
    cold_users.append(int(line.replace('\n','')))
    line = f.readline()
  f.close()

target_users_test = np.loadtxt("data/data_target_users_test.csv", dtype='int32', skiprows= 1)
import random
top100 = topPopRecommender.recommend(0, at=100)

top3 = list(top100[:3])
top95 = top100.tolist()
for i in range(5):
  top95.pop()

def cold_recommend():
  cold_items = topPopRecommender.recommend(0, at=5)
  cold_items = cold_items.tolist()
  random_items = random.sample(top95, 5)
  return  cold_items + random_items

In [44]:


outs = []
for user_id in target_users_test:
    Item_tuple =[]
    if user_id in cold_users:
      user = cold_To_warm_users.get(user_id)
      if user is None:
        Item_tuple = cold_recommend()
      else:
        Item_tuple = list(mix.recommend(user,10))[6:9] + cold_recommend()[0:7]
        print('user',user)
        print('item',Item_tuple)
    else:
      Item_tuple = mix.recommend(user_id,10)
    out = (user_id, Item_tuple)
    outs.append(out)

print(len(outs))


user 56
item [4346, 4693, 16376, 3805, 3565, 3572, 6526, 17955, 8638, 5113]
user 0
item [5562, 3569, 197, 6530, 3569, 6530, 10227, 17955, 8638, 5113]
user 88
item [9093, 5562, 5508, 16176, 5562, 9093, 16176, 17955, 8638, 5113]
user 40
item [5752, 5186, 1041, 18234, 4652, 8928, 8825, 17955, 8638, 5113]
user 72
item [8563, 8506, 8570, 4361, 8506, 3565, 3572, 17955, 8638, 5113]
user 16
item [9202, 9204, 9201, 15119, 9202, 9204, 9201, 17955, 8638, 5113]
user 28
item [13206, 12341, 3146, 14779, 2519, 11395, 13504, 17955, 8638, 5113]
user 56
item [4346, 4693, 16376, 3805, 3565, 3572, 6526, 17955, 8638, 5113]
user 16
item [9202, 9204, 9201, 15119, 9202, 9204, 9201, 17955, 8638, 5113]
user 117
item [6528, 8666, 8414, 8008, 6528, 8414, 9446, 17955, 8638, 5113]
user 56
item [4346, 4693, 16376, 3805, 3565, 3572, 6526, 17955, 8638, 5113]
user 72
item [8563, 8506, 8570, 4361, 8506, 3565, 3572, 17955, 8638, 5113]
user 73
item [5895, 3898, 8668, 8638, 3898, 5896, 8638, 17955, 8638, 5113]
user 75
item

In [0]:
target_users_test = np.loadtxt("data/data_target_users_test.csv", dtype='int32', skiprows= 1)
import random
top100 = topPopRecommender.recommend(0, at=100)
top95 = top100.tolist()
for i in range(5):
  top95.pop()

def cold_recommend():
  cold_items = topPopRecommender.recommend(0, at=5)
  cold_items = cold_items.tolist()
  random_items = random.sample(top95, 5)
  return  cold_items + random_items

def get_similar_train_user(uid):
  return 0

outs = []
most_popular = []
for user_id in target_users_test:
    if user_id in userList_unique:
        Item_tuple = mix.recommend(user_id,10)
    else:
      
      
    #recommend most popular top k
        Item_tuple = cold_recommend()
    out = (user_id, Item_tuple)
    outs.append(out)

In [0]:
with open('result_of_BPR4500+ItemKNNCF(6:4)+cold_user2.csv', 'w') as f:

    f.write('user_id,item_list\n')
    for out in outs:
        line = str(int(out[0]))+','
        for i in out[1]:
            line += str(i)+' '
        line += '\n'
        f.write(line)
    f.close()